In [ ]:
%scala
val BASE_DIR = "dbfs:/FileStore"
val DATA_DIR = "dbfs:/FileStore/text"
val data = spark.read
            .format("text")
            .option("lineSep",".")
            .load(DATA_DIR)

data.collect()

org.apache.spark.sql.SparkSession@bdf5c87BASE_DIR: String = dbfs:/FileStore
DATA_DIR: String = dbfs:/FileStore/text
data: org.apache.spark.sql.DataFrame = [value: string]
res16: Array[org.apache.spark.sql.Row] = Array([Structured Streaming is a scalable and fault-tolerant stream processing engine built on the Spark SQL engine], [ You can express your streaming computation the same way you would express a batch computation on static data], [ The Spark SQL engine will take care of running it incrementally and continuously and updating the final result as streaming data continues to arrive], [ You can use the Dataset/DataFrame API in Scala, Java, Python or R to express streaming aggregations, event-time windows, stream-to-batch joins, etc], [ The computation is executed on the same optimized Spark SQL engine], [ Finally, the system ensures end-to-end exactly-once fault-tolerance guarantees through checkpointing and Write-Ahead Logs], [ In short, Structured Streaming provides fast, scalable, fault-tolerant, end-to-end exactly-once stream processing without the user having to reason about streaming], [Internally, by default, Structured Streaming queries are processed using a micro-batch processing engine, which processes data streams as a series of small batch jobs thereby achieving end-to-end latencies as low as 100 milliseconds and exactly-once fault-tolerance guarantees], [ However, since Spark 2], [3, we have introduced a new low-latency processing mode called Continuous Processing, which can achieve end-to-end latencies as low as 1 millisecond with at-least-once guarantees], [ Without changing the Dataset/DataFrame operations in your queries, you will be able to choose the mode based on your application requirements], [In this guide, we are going to walk you through the programming model and the APIs], [ We are going to explain the concepts mostly using the default micro-batch processing model, and then later discuss Continuous Processing model], [ First, let’s start with a simple example of a Structured Streaming query - a streaming word count])

In [ ]:
# Refactor the code to class and functions for testing
class BatchWordCounts():

    def __init__(self):
        self.DATA_DIR = 'dbfs:/FileStore/test_data/text'


    def get_raw_data(self):
        
        from pyspark.sql.functions import explode, split
        text_data = spark.read \
                         .format('text') \
                         .option('lineSep', '.') \
                         .load(f'{self.DATA_DIR}')

        return text_data.select(explode(split(text_data.value, ' ')).alias('word'))
    

    def get_quality_data(self, raw_words):

        from pyspark.sql.functions import lower, trim

        return raw_words.select(lower(trim(raw_words.word)).alias('cleaned_words')) \
                        .where('cleaned_words is not null') \
                        .where("cleaned_words rlike '[a-z]'")

    def get_word_counts(self, quality_words):

        return quality_words.groupBy('cleaned_words').count()


    def overwrite_word_counts(self, word_counts):

        word_counts.write \
                   .format('delta') \
                   .mode('overwrite') \
                   .saveAsTable('word_counts')


    def execute(self):

        print(f'\tExecuting Word Count...', end='')

        raw_words = self.get_raw_data()
        quality_words = self.get_quality_data(raw_words)
        word_counts = self.get_word_counts(quality_words)
        self.overwrite_word_counts(word_counts)

        print(' Done.')
        

In [ ]:
class BatchWordCountsTestSuite():

    def __init__(self):
        self.BASE_DIR = 'dbfs:/FileStore'
        self.DATA_DIR = 'dbfs:/FileStore/test_data/text'


    def clean_up_for_testing(self):

        print('Starting cleaning...', end='')

        spark.sql('DROP TABLE IF EXISTS word_counts')
        dbutils.fs.rm('/user/hive/warehouse/word_counts', recurse=True)
        dbutils.fs.rm(f'{self.BASE_DIR}/checkpoint', recurse=True)
        dbutils.fs.rm(f'{self.BASE_DIR}/test_data/text/', recurse=True)
        dbutils.fs.mkdirs(f'{self.DATA_DIR}')

        print(' Done.')

    
    def get_data(self, file_num):

        print('Getting data...', end='')

        dbutils.fs.mkdirs(f'{self.BASE_DIR}/test_data/text/')
        dbutils.fs.cp(f'{self.DATA_DIR}/text_data_{file_num}.txt', 
                      f'{self.BASE_DIR}/test_data/text/')
        
        print(' Done.')

    
    def assert_result(self, expected_result):
        
        actual_result = spark.sql(
            '''
            SELECT SUM(count)
            FROM word_counts
            WHERE SUBSTR(cleaned_words, 1, 1) == 's'
            '''
        ).collect()[0][0]

        assert expected_result == actual_result, f'Test failed! Expected result is {expected_result}. Got {actual_result} instead.'


    def run_tests(self):

        self.clean_up_for_testing()
        word_counter = BatchWordCounts()

        expected_results = [25, 32, 37]
        for i in range(len(expected_results)):

            print(f'Testing file No.{i + 1}...')

            self.get_data(i + 1)
            word_counter.execute()
            self.assert_result(expected_results[i])

            print(f'File No.{i + 1} test completed.\n')


In [ ]:
batch_word_counts_tester = BatchWordCountsTestSuite()
batch_word_counts_tester.run_tests()

Starting cleaning... Done.
Testing file No.1...
Getting data... Done.
	Executing Word Count... Done.
+---------------+-----+
|  cleaned_words|count|
+---------------+-----+
|        ensures|    1|
|         stream|    2|
|fault-tolerance|    1|
|           will|    1|
|            you|    3|
|            can|    2|
|          java,|    1|
|     guarantees|    1|
|         arrive|    1|
|         system|    1|
|       provides|    1|
|            api|    1|
|  aggregations,|    1|
|             in|    2|
|           take|    1|
|           same|    2|
| fault-tolerant|    1|
|      continues|    1|
|      scalable,|    1|
|          fast,|    1|
+---------------+-----+
only showing top 20 rows

File No.1 test completed.

Testing file No.2...
Getting data... Done.
	Executing Word Count... Done.
+---------------+-----+
|  cleaned_words|count|
+---------------+-----+
|        ensures|    1|
|         stream|    2|
|fault-tolerance|    2|
|           will|    2|
|            you|    4|
|   